## Loading Libraries & Dataframe

In [ ]:
# Core modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import re
import joblib
import os

# Scipy & Statsmodels
from scipy import stats
from scipy.stats import skew, boxcox, mstats, randint
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.regressionplots import plot_leverage_resid2

# Scikit-Learn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# XGBoost
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance

# LightGBM
import lightgbm as lgb
from lightgbm import LGBMRegressor

# Scikit-Optimize
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

In [ ]:
path_df = pd.read_csv(r"laptop_prices.csv")
path_df

## Transformers

In [ ]:
from custom_transformers import (
    CPUSeriesExtractor,
    GPUSeriesExtractor,
    CardinalityReducer,
    PixelCalculator,
    LogTransformer,
    ColumnDropper,
    KMeansClusterAdder
)

## A quick overview **(EDA)**

In [ ]:
path_df.columns

In [ ]:
path_df.info()

No missing values in the dataset

In [ ]:
msno.matrix(path_df)

No duplicates in dataset

In [ ]:
path_df.duplicated().sum()

### ~`Company`
Column seems to have a high cardinality between it's results due to companies having bigger market shares

In [ ]:
company_names = path_df["Company"].unique()
print(f"Laptop companies: {company_names}")

plt.figure(figsize= (18,12))

company_counts = path_df["Company"].value_counts()
avg_company_count = company_counts.mean()

# List comprehension for color condition
colors = ["green" if count >= avg_company_count else "red" for count in company_counts]

sns.countplot(x = "Company",
              data = path_df,
              order = company_counts.index,
              palette = colors,
              edgecolor = "black")
plt.title("Market shares of laptop companies")
# Average count line
plt.axhline(y = avg_company_count,
            color = "black",
            linestyle = "--",
            label = f"Mean: {avg_company_count:.0f}")
plt.legend()
plt.show()

### `Product`
Column has repeated entries (nothing concerning)

In [ ]:
path_df["Product"].nunique()

### ~`TypeName`
**Notebooks** seem to be the most popular

In [ ]:
unique_types = path_df["TypeName"].nunique()
print(f"Laptop types: {unique_types}")

plt.figure(figsize = (18,12))

typecounts = path_df["TypeName"].value_counts()

sns.countplot(x = "TypeName",
              data = path_df,
              palette = "Set2",
              edgecolor = "black",
              order = typecounts.index)
plt.title("Laptop type popularity")
plt.xlabel("Laptop Type")
plt.show()

### `Inches`
Column seemed to have multiple high leverage points that after a deep research it was concluded that they weren't outliers. They were small sized due to them mostly being convertibles or small netbooks

In [ ]:
avg_size = path_df["Inches"].mean()
print(f"Average laptop size: {avg_size:.0f} inches")

Q1 = path_df["Inches"].quantile(0.25)
Q3 = path_df["Inches"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = path_df[(path_df["Inches"] < lower) | (path_df["Inches"] > upper)]
print(f"Number of outliers: {len(outliers)}")
print(f"Outlier bounds: [{lower:.2f}, {upper:.2f}]")
display(outliers)

plt.figure(figsize = (18,12))
sns.boxplot(x = "Inches",
            data = path_df,
            color = "grey")
plt.show()

### `Ram`
Just like last column. `Ram` Also had high leverage points in terms of ram capacity beihng above the average in some cases. That was due to these laptops being a high performance **gaming** or **workstation** laptops

In [ ]:
avg_ram = path_df["Ram"].mean()
print(f"Average laptop ram: {avg_ram:.0f} GB")

Q1 = path_df["Ram"].quantile(0.25)
Q3 = path_df["Ram"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = path_df[(path_df["Ram"] < lower) | (path_df["Ram"] > upper)]
print(f"Number of outliers: {len(outliers)}")
print(f"Outlier bounds: [{lower:.2f}, {upper:.2f}]")
display(outliers)

plt.figure(figsize = (18,12))
sns.boxplot(x = "Ram",
            data = path_df,
            color = "grey")
plt.show()

### ~`OS`
We see high cardinality of categorical variables with a large dominance of windows 10 being the main OS for most laptops

In [ ]:
os_counts = path_df["OS"].value_counts()
os_avg = os_counts.mean()
os_num = path_df["OS"].nunique()
print(f"There are a total of {os_num} operating systems in the market")

color = ["green" if count > os_avg else "red" for count in os_counts]

plt.figure(figsize = (18,12))
sns.countplot(x = "OS",
              data = path_df,
              palette = color,
              order = os_counts.index,
              edgecolor = "black")
plt.axhline(y = os_avg,
            color = "black",
            linestyle = "--",
            label = f"Mean: {os_avg:.0f}")
plt.legend()
plt.title("Preinstalled operating systems in laptops")
plt.show()

### `Weight`
There seems to be a couple influencial entires where they weigh more than 2.71 KG but it's to be expected due to their bigger surface area compared to other laptops

In [ ]:
avg_weight = path_df["Weight"].mean()
print(f"Average weight: {avg_weight:.2f} KG")

Q1 = path_df["Weight"].quantile(0.25)
Q3 = path_df["Weight"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = path_df[(path_df["Weight"] < lower) | (path_df["Weight"] > upper)]
print(f"Number of outliers: {len(outliers)}")
print(f"Outlier bounds: [{lower:.2f}, {upper:.2f}]")
display(outliers)

plt.figure(figsize = (18,12))
sns.boxplot(x = "Weight",
            data = path_df,
            color = "grey")
plt.show()

As we see, there's a positive linear relationship between the laptop size in inches and its weight

In [ ]:
plt.figure(figsize = (12,10))
sns.regplot(x = "Inches",
                y = "Weight",
                data = path_df,
                scatter_kws = {"color" : "grey", "alpha" : 0.8},
                line_kws = {"color" : "red", "linewidth" : 2})
plt.title("Relationship between laptop size & weight")
plt.show()

### `Price_euros`
Price seems to be skewed which will need transformation to adjust and normalize or checking outliers

In [ ]:
def state_skew(x):
    if abs(x) > 1:
        return "Very skewed"
    elif 0.5 <= abs(x) <= 1:
        return "Moderately skewed"
    else : 
        return "Symmetrical"
def state_kurtosis(x):
    if x > 1:
        return "Leptokurtic (Many Outliers)"
    elif x < -1:
        return "Playtukurtic (Few Outliers)"
    else :
        return "Mesokurtic (Normal tails)"
    
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (12, 10))
sns.histplot(x = "Price_euros",
             data = path_df,
             color = "grey",
             edgecolor = "black",
             kde = True,
             ax = ax1)
ax1.set_title("Checking column normality")
sns.regplot(x = "Price_euros",
                y = "Weight",
                data = path_df,
                scatter_kws = {"color" : "grey", "alpha" : 0.8},
                line_kws = {"color" : "red", "linewidth" : 2},
                ax = ax2)
ax2.set_title("Relationship of price with weight")
plt.show()

price_kurtosis = path_df["Price_euros"].kurtosis()
price_skew = path_df["Price_euros"].skew()
print(f"Price kurtosis before transformation: {price_kurtosis:.2f}, {state_kurtosis(price_kurtosis)}")
print(f"Price skew before transformation: {price_skew:.2f}, {state_skew(price_skew)}")

From what we can see is there are multiple outliers that could affect the modeling phase.
After further investigation they could seem dangerous but after transformation most likely it'll be completely normalized due to these highly priced laptops being either workstation, ultrabooks or gaming laptops that have alot of expensive parts

In [ ]:
avg_price = path_df["Price_euros"].mean()
print(f"Average price: {avg_price:.2f}€")

Q1 = path_df["Price_euros"].quantile(0.25)
Q3 = path_df["Price_euros"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q1 + 1.5 * IQR

outliers = path_df[(path_df["Price_euros"] < lower) | (path_df["Price_euros"] > upper)]
print(f"Number of outliers: {len(outliers)}")
print(f"Outlier bounds: [{lower:.2f}, {upper:.2f}]")
display(outliers)

plt.figure(figsize = (12, 10))
sns.boxplot(x = "Price_euros",
            data = path_df,
            color = "grey")
plt.show()

### ~`Screen`, `ScreenW`, & `ScreemH`
Dominance of **Full HD** & **Standard** over the laptop market (High Cardinality)

In [ ]:
screen_count = path_df["Screen"].value_counts()

plt.figure(figsize = (10,8))
plt.pie(screen_count.values,
        colors = sns.color_palette("Set2"),
        autopct = "%1.1f%%",
        explode = [0, 0.1, 0, 0])  # Shows percentage
plt.title("Laptop screens by popularity")
plt.legend(labels = screen_count.index,
           title = "Screen type",
           loc = "best")
plt.show()

**1920x1080p** dominated laptop screens and to this day it's the most popular and most used resolution although **2560x1440p** & **3840x2160p** (also known as **4K**) has been getting more popular in the recent years

In [ ]:
# Generating the full screen resolution
path_df["Resolution"] = path_df["ScreenW"].astype(str) + "x" + path_df["ScreenH"].astype(str)
res_count = path_df["Resolution"].value_counts()
avg_res = res_count.mean()

color = ["green" if count >= avg_res else "red" for count in res_count]

plt.figure(figsize = (18, 12))
sns.countplot(x = "Resolution",
              data = path_df,
              order = res_count.index,
              palette = color,
              edgecolor = "black")
plt.title("Most popular laptop resolutions")
plt.axhline(y = avg_res,
            color = "black",
            linestyle = "--",
            label = f"Mean: {avg_res:.0f}")
plt.legend()
plt.show()

### ~`Touchscreen`

In [ ]:
touch_count = path_df["Touchscreen"].value_counts()

plt.figure(figsize = (10, 8))
plt.pie(touch_count.values,
        colors = sns.color_palette("Set2"),
        autopct = "%1.1f%%",
        explode = [0, 0.1])
plt.title("Touchscreen laptop proportion")
plt.legend(labels = touch_count.index,
           title = "Touchscreen ?",
           loc = "best")

### ~`IPSpanel`

In [ ]:
ips_count = path_df["IPSpanel"].value_counts()

plt.figure(figsize = (10, 8))
plt.pie(ips_count.values,
        colors = sns.color_palette("Set2"),
        autopct = "%1.1f%%",
        explode = [0, 0.1])
plt.title("IPS screens laptop proportion")
plt.legend(labels = ips_count.index,
           title = "IPS ?",
           loc = "best")

### ~`RetinaDisplay`
Very little laptops had retina display screens where this technology was mostly unique to apple macbooks

In [ ]:
retina_count = path_df["RetinaDisplay"].value_counts()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (10,8))
ax2.pie(retina_count.values,
        colors = sns.color_palette("Set2"),
        autopct = "%1.1f%%")
ax2.legend(labels = retina_count.index,
           title = "Retina ?",
           loc = "upper right")
ax2.set_title("Retina display laptop proportion")

# Showing only companies making retina display laptops
retina_by_company = path_df[path_df["RetinaDisplay"] == "Yes"].groupby("Company")["RetinaDisplay"].count().sort_values(ascending = False)
retina_companies = retina_by_company[retina_by_company > 0]

sns.barplot(x = retina_companies.index,
            y = retina_companies.values,
            color = "grey",
            edgecolor = "black",
            ax = ax1)
ax1.set_title("Retina display by company")
ax1.tick_params(axis = "x", rotation = 45)
plt.tight_layout()
plt.show()

print(f"Only {retina_companies.shape[0]} companies have retina display laptops")
print(f"Total Retina laptops: {retina_count.get(1, 0)} ({retina_count.get(1, 0)/len(path_df)*100:.1f}%)")

### ~`CPU_company`
Intel holds a massive market share in CPUs for laptops

In [ ]:
plt.figure(figsize = (12, 10))
sns.countplot(x = "CPU_company",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Laptop cpu companies market shares")
plt.show()

### `CPU_freq`
Although **Intel** holds the bigger marketshare. **AMD** seems to have the better performing CPUs on average

In [ ]:
cpu_company_freq = path_df.groupby("CPU_company")["CPU_freq"].mean().sort_values(ascending = False).round(1)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (12, 10))
sns.histplot(x = "CPU_freq",
             data = path_df,
             color = "grey",
             edgecolor = "black",
             kde = True,
             ax = ax1)
ax1.set_title("Laptop CPU clock speed distribution")
ax1.set_ylabel("Frequency (GHz)")

sns.barplot(x = cpu_company_freq.index,
            y = cpu_company_freq.values,
            palette = "Set2",
            edgecolor = "black",
            ax = ax2)
ax2.set_title("Average CPU clock speed per company")
ax2.set_ylabel("Frequency (GHz)")
for i, v in enumerate(cpu_company_freq.values):
    ax2.text(i, v - 0.15, f"{v:.1f}GHz", ha = "center", fontweight = "bold")
plt.show()

### ~`CPU_model`
There is a massive popularity of "**Core i**" models compared to other models of **Intel** and other CPU companies (High Cardinality)

In [ ]:
# Extracting 
def extract_cpu_model(cpu_string):
    patterns = [
        r'(Core [im]\d)',           # Core i3, i5, i7, i9, m3, m5, etc...
        r'(Ryzen \d)',              # Ryzen 3, 5, 7
        r'(Atom [xX]\d)',           # Atom x5, x7
        r'(Celeron)',               # Celeron
        r'(Pentium)',               # Pentium
        r'(Xeon)',                  # Xeon
        r'(A\d+)',                  # AMD A6, A9, A10, A12, etc...
        r'(E2)',                    # AMD E2
        r'(FX)',                    # AMD FX
    ]
    
    for pattern in patterns:
        match = re.search(pattern, cpu_string, re.IGNORECASE)
        if match:
            return match.group(1)
    return "Other"

path_df["CPU_series"] = path_df["CPU_model"].apply(extract_cpu_model)
series_counts = path_df["CPU_series"].value_counts()
avg_series = series_counts.mean()

colors = ["green" if count > avg_series else "red" for count in series_counts]

plt.figure(figsize = (16,10))
sns.countplot(x = "CPU_series",
              data = path_df,
              palette = colors,
              order = series_counts.index,
              edgecolor = "black")
plt.axhline(y = avg_series,
            color = "black",
            linestyle = "--",
            label =f"Mean: {avg_series:.0f}")
plt.legend()
plt.title("CPU models popularity in laptops")
plt.show()

### `PrimaryStorage`
Most laptops had **256GBs** as a primary drive that had their operating system on

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "PrimaryStorage",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Primary storage popularity in GBs")
plt.xlabel("GBs")
plt.show()

### `SecondaryStorage`
Most laptops don't have any secondary hard drives installed which could result in an imbalance in modeling later on

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "SecondaryStorage",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Secondary storage popularity in GBs")
plt.xlabel("GBs")
plt.show()

### ~`PrimaryStorageType`
**SSDs** generally have been the most popular storage type due to their fast read and write frequency & longevity

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "PrimaryStorageType",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Primary storage type popularity")
plt.xlabel("Storage Type")
plt.show()

### ~`SecondaryStorageType`
Surprisingly laptops that do have a seconadry storage installed usually use **HDD** which means that usually use secondary storage to store important files and don't need to use the massive read & write speeds that an **SSD** provides

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "SecondaryStorageType",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Secondary storage type popularity")
plt.xlabel("Storage Type")
plt.show()

### ~`GPU_company`
**Intel** has the largest market share which is expected due to them winning by a landslide in CPUs as well

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "GPU_company",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Laptop GPU companies market share")
plt.xlabel("GPU Companies")
plt.show()

### ~`GPU_model`
**Intel integrated graphics** had the highest market share due to them being present in most budget to medium priced laptop without a dedicated GPU

In [ ]:
# Extracting GPU series
def extract_gpu_model(gpu_string):
    gpu_string = str(gpu_string)
    
    # NVIDIA patterns
    if re.search(r'GTX', gpu_string, re.IGNORECASE):                    # GTX
        return "GeForce GTX"
    elif re.search(r'MX\d{3}', gpu_string, re.IGNORECASE):              # MX
        return "GeForce MX"
    elif re.search(r'GeForce \d{3}', gpu_string, re.IGNORECASE):        # GeForce
        return "GeForce (Low-end)"
    elif re.search(r'Quadro', gpu_string, re.IGNORECASE):               # Quadro
        return "Quadro"
    
    # AMD patterns
    elif re.search(r'Radeon Pro', gpu_string, re.IGNORECASE):           # Radeon Pro
        return "Radeon Pro"
    elif re.search(r'Radeon RX', gpu_string, re.IGNORECASE):            # Radeon RX
        return "Radeon RX"
    elif re.search(r'Radeon R\d', gpu_string, re.IGNORECASE):           # Radeon R
        return "Radeon R"
    elif re.search(r'Radeon', gpu_string, re.IGNORECASE):               # Radeon 530, etc...
        return "Radeon (Other)"
    
    # Intel patterns
    elif re.search(r'UHD Graphics', gpu_string, re.IGNORECASE):         # UHD Graphics 550, etc...
        return "Intel UHD"
    elif re.search(r'Iris Plus', gpu_string, re.IGNORECASE):            # Iris
        return "Intel Iris Plus"
    elif re.search(r'Iris Pro', gpu_string, re.IGNORECASE):             # Iris Pro
        return "Intel Iris Pro"
    elif re.search(r'Iris', gpu_string, re.IGNORECASE):                 # Iris
        return "Intel Iris"
    elif re.search(r'HD Graphics', gpu_string, re.IGNORECASE):          # HD Graphics
        return "Intel HD"
    
    return "Other"

path_df["GPU_series"] = path_df["GPU_model"].apply(extract_gpu_model)
gpu_series_counts = path_df["GPU_series"].value_counts()
avg_gpu_series = gpu_series_counts.mean()

colors = ["green" if count > avg_gpu_series else "red" for count in gpu_series_counts]

plt.figure(figsize = (12, 10))
sns.countplot(x = "GPU_series",
              data = path_df,
              palette = colors,
              order = gpu_series_counts.index,
              edgecolor = "black")
plt.axhline(y = avg_gpu_series,
            color = "black",
            linestyle = "--",
            label = f"Mean: {avg_gpu_series:.0f}")
plt.legend()
plt.title("GPU series popularity in laptops")
plt.xlabel("GPU Series")
plt.xticks(rotation = 45, ha = "right")
plt.tight_layout()
plt.show()

### Correlation Matrix (Before Transformation)

In [ ]:
num_df = path_df.select_dtypes("number")
plt.figure(figsize = (12,10))
sns.heatmap(num_df.corr(),
            annot = True,
            cmap = "Greens",
            fmt = ".2f",
            linewidths = 0.2)
plt.title("Correlation Matrix (Before)")
plt.show()

## Choosing the variables (Feature Engineering)

### Pipeline configuration

In [ ]:
# Transformations & encoding groups
te_cols = ["Company", "CPU_series", "GPU_series"]

ohe_cols = ['TypeName', 'OS', 'Screen', 'CPU_company', 
            'PrimaryStorageType', 'SecondaryStorageType', 'GPU_company']

binary_cols = ["Touchscreen", "IPSpanel", "RetinaDisplay"]

log_cols = ["PrimaryStorage", "SecondaryStorage"]

# Columns getting dropped
drop_cols = ['Product', 'CPU_model', 'GPU_model', 'ScreenW', 'ScreenH', 'Resolution']

# Scaling groups
robust_features = ['Ram', 'PrimaryStorage', 'SecondaryStorage']
standard_features = ['Inches', 'CPU_freq', 'Pixels', "Weight"]

### Pipeline setup

In [ ]:
# Complete pipeline with any regressor, auto log1p/expm1 on price.
def build_pipeline(model, n_clusters = 4):

    return Pipeline([
        # Feature engineering
        ("cpu_extract", CPUSeriesExtractor()),
        ("gpu_extract", GPUSeriesExtractor()),
        ("pixels", PixelCalculator(log_transform=True)),
        ("log_storage", LogTransformer(columns=log_cols)),
        ("cardinality", CardinalityReducer(
            columns=te_cols,
            threshold="mean",
            exceptions={"Company": ["Apple"]}
        )),
        ("drop", ColumnDropper(columns = drop_cols)),
        
        # Encoding & scaling
        ("encoding_scaling", ColumnTransformer(transformers =[
            ("te", TargetEncoder(cv = 5, smooth= "auto"), te_cols), # Has built in auto smoothing 
            ("ohe", OneHotEncoder(handle_unknown = "ignore", sparse_output = False, drop = "first"), ohe_cols),
            ("binary", OrdinalEncoder(), binary_cols),
            ("robust", RobustScaler(), robust_features),
            ("std", StandardScaler(), standard_features)
        ], remainder = "passthrough")),
        
        # Clustering
        ("cluster", KMeansClusterAdder(n_clusters = n_clusters)),
        
        # Log transformations
        ("model", TransformedTargetRegressor(
            regressor = model,
            func = np.log1p,
            inverse_func = np.expm1
        ))
    ])

# Complete model hyperparameter tuning, training, & testing, outputing metrics
def tune_and_evaluate(name, model, param_config, X_train, X_test, y_train, y_test):
    pipe = build_pipeline(model)
    
    # Filters parameter labeling from pipeline
    prefixed_params = {
        f"model__regressor__{k}": v for k, v in param_config["params"].items()
    }
    
    if not prefixed_params:
        pipe.fit(X_train, y_train)
        best_pipe = pipe
        best_params_clean = {}
        
    elif param_config["method"] == "random": # RandomSearchCV()
        rand_search = RandomizedSearchCV(
            estimator = pipe,
            param_distributions = prefixed_params,
            n_iter = param_config.get("n_iter", 100),
            cv = 10,
            scoring = "neg_mean_absolute_error",
            n_jobs = -1,
            verbose = 1,
            random_state = 90
        )
        rand_search.fit(X_train, y_train)
        best_pipe = rand_search.best_estimator_
        best_params_clean = {k.replace("model__regressor__", ""): v for k, v in rand_search.best_params_.items()}
        
    elif param_config["method"] == "grid": # GridSearchCV()
        grid_search = GridSearchCV( 
            estimator = pipe,
            param_grid = prefixed_params,
            cv = 10,
            scoring = "neg_mean_absolute_error",
            n_jobs = -1,
            verbose = 1
        )
        grid_search.fit(X_train, y_train)
        best_pipe = grid_search.best_estimator_
        best_params_clean = {k.replace("model__regressor__", ""): v for k, v in grid_search.best_params_.items()}
        
    else:
        bay_search = BayesSearchCV(
            estimator = pipe,
            search_spaces = prefixed_params,
            cv = 10,
            scoring = "neg_mean_absolute_error",
            n_jobs = -1,
            verbose = 0,
            random_state = 72
        )
        bay_search.fit(X_train, y_train)
        best_pipe = bay_search.best_estimator_
        best_params_clean = {k.replace("model__regressor__", ""): v for k, v in bay_search.best_params_.items()}
        
    y_pred = best_pipe.predict(X_test)
    
    results = {
        "Model": name,
        "  R2": r2_score(y_test, y_pred),
        " MAE": mean_absolute_error(y_test, y_pred),
        "RMSE": root_mean_squared_error(y_test, y_pred)
    }
    
    print(f"\n===={name} results====\n")
    if best_params_clean:
        print(f"    Method: {param_config['method'].upper()}")
        print(f"    Best parameters: {best_params_clean}")
    for metric, value in results.items():
        if metric != "Model":
            print(f"    {metric}: {value:.2f}")
    print("\n--------------------------\n")
    return results, best_pipe

## Setting up data for modeling

In [ ]:
path_df = pd.read_csv(r"laptop_prices.csv")

X = path_df.drop(columns = "Price_euros")
y = path_df["Price_euros"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 50, test_size = 0.2)

print(f"Training on {X_train.shape[0]} laptops.")
print(f"Testing on {X_test.shape[0]} laptops.")

## Model Setup
XGBoost model wins by a significant margin compared to other models in terms of **R2**, **MAE**, & **RMSE** 

In [ ]:
# LinearRegression()
lr_results, lr_model_pipe = tune_and_evaluate(
    "Linear Regression", LinearRegression(),
    {"method": None, "params": {}}, # LR is non hyperparametic
    X_train, X_test, y_train, y_test    
)

# ElasticNetCV()
en_results, en_model_pipe = tune_and_evaluate(
    "Elastic Net", ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
                                cv = 5,
                                random_state = 5,
                                alphas = np.logspace(-4, 1, 100),  # force search over smaller alphas
                                max_iter = 10000),
    {"method": None, "params": {}},
    X_train, X_test, y_train, y_test
)


# RandomForestRegressor()
rfr_results, rfr_model_pipe = tune_and_evaluate(
    "Random Forest Regressor", RandomForestRegressor(random_state = 42),
    {
        "method": "random",
        "n_iter": 100,
        "params": { # Model Parameters
            "n_estimators": randint(100, 500), # number of trees
            "max_depth": [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, None], # maximum number of levels
            "min_samples_split": randint(2, 10), # minimum number of samples required to split a node
            "max_features": ["sqrt", "log2", 1.0] # number of features to consider on every split
        }
    },
    X_train, X_test, y_train, y_test
)

# XGBoost
xgb_results, xgb_model_pipe = tune_and_evaluate(
    "XGBoost", XGBRegressor(
        objective = "reg:squarederror", # focusing on minimizing squared error
        tree_method = "hist", # histogram binning trees for higher speeds
        random_state = 67,
        n_jobs = -1 # full powaaaaaaaaa
    ),
    {"method": "grid",
     "n_iter": 100,
     "params": {
        "n_estimators": [1000, 1100, 1150, 1200],
        "max_depth": [5, 6, 7],
        "eta": [0.01, 0.02 , 0.03], # learning rate
        "subsample": [0.5, 0.6, 0.7], # prevents overfitting by using % of the testing rows per tree
        "colsample_bytree": [0.8, 0.7, 0.9] # same as subsampling but hides features (RF feature bagging)
     }
    },
    X_train, X_test, y_train, y_test
)

# LightGBM
lgbm_results, lgbm_model_pipe = tune_and_evaluate(
    "LightGBM", LGBMRegressor(
        random_state= 67,
        n_jobs = -1,
    ),
    {"method": "baysian", # Using baysian search due being more compatible 
     "n_iter": 50, # baysian doesn't need 100
     "params": {
         "objective": Categorical(["regression"]),
         "boosting_type": Categorical(["dart", "gbdt"]), # dart boosing is effective but takes alot of time due to the larger processing load
         "max_depth": Integer(3, 6),
         "learning_rate": Real(0.1, 0.3), # eta
         "num_leaves": Integer(100, 200), 
         "n_estimators": Integer(900, 1500)
     }
            
    },
    X_train, X_test, y_train, y_test
        )

comparison = pd.DataFrame([lr_results, en_results, rfr_results, xgb_results, lgbm_results])
comparison.sort_values("  R2", ascending = False).round(2)

### Linearity Assumptions

In [ ]:
X_test_transformed = lr_model_pipe[:-1].transform(X_test)

y_pred = lr_model_pipe.predict(X_test)
residuals = y_test - y_pred

# Visual diagnostics
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Linearity
axes[0, 0].scatter(y_pred, residuals, alpha=0.5, color="grey", edgecolor="black")
axes[0, 0].axhline(y=0, color="red", linestyle="--", linewidth=2)
axes[0, 0].set_xlabel("Predicted Values")
axes[0, 0].set_ylabel("Residuals")
axes[0, 0].set_title("1. Linearity: Residuals vs Predicted")

# 2. Normality of Residuals
sm.qqplot(residuals, line="45", ax=axes[0, 1], markerfacecolor="grey", markeredgecolor="black")
axes[0, 1].set_title("2. Normality: Q-Q Plot of Residuals")

# 3. Homoscedasticity
standardized_residuals = (residuals - residuals.mean()) / residuals.std()
axes[1, 0].scatter(y_pred, np.sqrt(np.abs(standardized_residuals)), alpha=0.5, color="grey", edgecolor="black")
axes[1, 0].set_xlabel("Predicted Values")
axes[1, 0].set_ylabel("√|Standardized Residuals|")
axes[1, 0].set_title("3. Homoscedasticity: Scale-Location Plot")

# 4. Residuals Distribution
sns.histplot(residuals, kde=True, color="grey", edgecolor="black", ax=axes[1, 1])
axes[1, 1].axvline(x=0, color="red", linestyle="--", linewidth=2)
axes[1, 1].set_title("4. Residuals Distribution")

plt.tight_layout()
plt.show()

# Statistical assumptions
print(f"\n{'='*50}")
print("Assumption Test Results")
print(f"{'='*50}")

# Shapiro-Wilk Test
sample_residuals = residuals.sample(min(500, len(residuals)), random_state=42) if len(residuals) > 500 else residuals
shapiro_stat, shapiro_p = stats.shapiro(sample_residuals)
print(f"\n1. Normality (Shapiro-Wilk Test):")
print(f"   Statistic: {shapiro_stat:.4f}, p-value: {shapiro_p:.4f}")
print(f"   Result: {'✓ Normal' if shapiro_p > 0.05 else '✗ Not Normal'} (α=0.05)")

# Durbin-Watson Test
# We use .values to ensure we pass a clean array to avoid indexing bugs
dw_stat = durbin_watson(residuals.values)
print(f"\n2. Independence (Durbin-Watson Test):")
print(f"   Statistic: {dw_stat:.4f}")
print(f"   Result: {'✓ No autocorrelation' if 1.5 < dw_stat < 2.5 else '✗ Possible autocorrelation'}")

# Breusch-Pagan Test 
# CRITICAL: Using X_test_transformed here ensures all features are numeric
X_test_with_const = sm.add_constant(X_test_transformed)
bp_stat, bp_p, _, _ = het_breuschpagan(residuals, X_test_with_const)
print(f"\n3. Homoscedasticity (Breusch-Pagan Test):")
print(f"   Statistic: {bp_stat:.4f}, p-value: {bp_p:.4f}")
print(f"   Result: {'✓ Homoscedastic' if bp_p > 0.05 else '✗ Heteroscedastic'} (α=0.05)")

# Model Performance
print(f"\n{'='*50}")
print("Model Performance Metrics")
print(f"{'='*50}")
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")

### Correlation Matrix (After transformation)

In [ ]:
data_engine = xgb_model_pipe[:-1]

X_transformed = data_engine.transform(X_train)

coltrans = xgb_model_pipe.named_steps["encoding_scaling"]
feature_names = list(coltrans.get_feature_names_out()) + ["Cluster"]

df_corr = pd.DataFrame(X_transformed, columns=feature_names)
df_corr["Price_euros"] = y_train.values

plt.figure(figsize=(18, 14))
sns.heatmap(df_corr.corr(),
            annot=False,
            cmap="Greens",
            linewidths=0.5)
plt.title("Correlation Matrix (After)")
plt.show()

## Feature Importance
Although it would seem that some features are redundant looking at the chart but the numbers say otherwise, keeping these features although they don't deliver significant importance random forest uses feature bagging which uses different combinations which could result in better accuracy overall

In [ ]:
# Access the actual model inside the pipeline
xgb_core = xgb_model_pipe.named_steps["model"].regressor_
coltrans = xgb_model_pipe.named_steps["encoding_scaling"]
cluster = xgb_model_pipe.named_steps["cluster"]

feature_names = cluster.get_feature_names_out(coltrans.get_feature_names_out())
importances = xgb_core.feature_importances_

feature_importance = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values(by = "importance", ascending = False)

plt.figure(figsize = (12,10))
sns.barplot(x = "importance",
            y = "feature",
            data = feature_importance,
            palette = "Set2",
            edgecolor = "black")
plt.title("Feature importance for model")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

## Identifying our K
Using **Elbow plot** to identify the best K for clustering

In [ ]:
pre_cluster_pipe = xgb_model_pipe[: -2] # Setting up pipeline without the y_transformation to for clustering set up

X_transformed = pre_cluster_pipe.transform(X_train) # Scaling & encoding data

inertia = []

for k in range(1, 10):
    km = KMeans(n_clusters = k, 
                random_state = 42,
                n_init = 10) # Clusters 10 times to avoid bad clusterings
    km.fit_transform(X_transformed)
    inertia.append(km.inertia_)
    
plt.figure(figsize = (12, 10))
sns.lineplot(x = range(1, 10),
                y = inertia,
                marker = "o",
                color = "red")
plt.xlabel("K")
plt.ylabel("Intertia")
plt.title("Elbow Plot")
plt.show()
print("Best K = 4")

Now I'll setup the clustering algorithm in the pipeline

In [ ]:
labels = cluster.kmeans_.predict(X_transformed)

# Attaching Cluster column back to the training dataset
cluster_df = X_train.copy()
cluster_df["Cluster"] = labels
cluster_df["Price"] = y_train.values

# Visualizing clusters
pca = PCA(n_components = 2, random_state = 42)
X_2d = pca.fit_transform(X_transformed)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 8))

scatter1 = ax1.scatter(X_2d[:, 0],
                       X_2d[:, 1],
                       c = labels,
                       cmap = "Set2",
                       alpha = 0.6,
                       edgecolor = "black",
                       linewidth = 0.3)
ax1.set_title("Laptop Clusters (PCA Projection)")
ax1.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)")
ax1.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)")
ax1.legend(*scatter1.legend_elements(), title = "Cluster")

scatter2 = ax2.scatter(X_2d[:, 0],
                       X_2d[:, 1],
                       c = y_train.values,
                       cmap = "RdYlGn",
                       alpha = 0.6,
                       edgecolor = "black",
                       linewidth = 0.3)
ax2.set_title("Price Distribution (PCA Projection)")
ax2.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)")
ax2.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)")
plt.colorbar(scatter2, ax = ax2, label = "Price (€)")

plt.tight_layout()
plt.show()

## Business Insights

Based on the cluster analysis, laptops naturally group into distinct market segments:

- **Cluster 0** — Budget laptops: Low RAM, basic CPUs, smaller screens
- **Cluster 1** — Mid-range notebooks: Standard specs, most popular segment  
- **Cluster 2** — Premium ultrabooks: High resolution, lightweight, premium pricing
- **Cluster 3** — Performance/Gaming: High RAM, powerful CPUs, heaviest & most expensive

In [ ]:
# Price breakdown per cluster
price_summary = cluster_df.groupby("Cluster")["Price"].agg(
    ["mean", "median", "count", "min", "max"]
).round(2)
price_summary.columns = ["Mean €", "Median €", "Count", "Min €", "Max €"]
print("                                                                    === Price per Cluster ===")
display(price_summary)

# Hardware profile per cluster
hardware_summary = cluster_df.groupby("Cluster").agg(
    Avg_Ram = ("Ram", "mean"),
    Avg_Weight = ("Weight", "mean"),
    Avg_Inches = ("Inches", "mean"),
    Avg_CPU_freq = ("CPU_freq", "mean"),
    Avg_Storage = ("PrimaryStorage", "mean"),
    Top_Type = ("TypeName", lambda x: x.mode()[0]),
    Top_Company = ("Company", lambda x: x.mode()[0]),
    Top_OS = ("OS", lambda x: x.mode()[0])
).round(2)
print("\n                                                                === Hardware Profile per Cluster ===")
display(hardware_summary)

# Cluster size distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (16, 6))

cluster_df.groupby("Cluster")["Price"].mean().plot(kind = "bar",
                                                   color = sns.color_palette("Set2"),
                                                   edgecolor = "black",
                                                   ax = ax1)
ax1.set_title("Average Price per Cluster")
ax1.set_ylabel("Price (€)")
ax1.tick_params(axis = "x", rotation = 0)

cluster_df["Cluster"].value_counts().sort_index().plot(kind = "pie",
                                                       colors = sns.color_palette("Set2"),
                                                       autopct = "%1.1f%%",
                                                       ax = ax2)
ax2.set_title("Cluster Size Distribution")
ax2.set_ylabel("")

plt.tight_layout()
plt.show()

# Model deployment
Dumping the models into a folder for later use in dashboard

In [ ]:
os.makedirs("models", exist_ok = True)

models = {
    "linear_regression": lr_model_pipe,
    "elastic_net": en_model_pipe,
    "random_forest_regressor": rfr_model_pipe,
    "xgboost": xgb_model_pipe,
    "lightgbm": lgbm_model_pipe
}

for name, pipe in models.items():
    joblib.dump(pipe, f"models/{name}_pipe.joblib")
    print(f"✅ Saved {name} → models/{name}_pipe.joblib")
    
joblib.dump(comparison, "models/comparison.joblib")
joblib.dump({"X_test": X_test, "y_test": y_test}, "models/test_data.joblib")

print("\nAll models saved to ./models/ successfully")